## Integrating a simple harmonic oscillator and trying to infer the spring constant

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline
import scipy.optimize as so
%matplotlib inline
import autograd.numpy as np  # Thinly-wrapped numpy
from autograd import grad  
import tensorflow as tf
from tensorflow.contrib import autograph
import tfleapfrog_copy2 as tflf

In [2]:
tf.VERSION

'1.10.0'

In [3]:
%load_ext autoreload
%autoreload 2

##  Define python functions first to compare with TF to debug 

In [4]:
def genData(x, v, npoints, std_noise_x, std_noise_v):
    noise_x = np.random.normal(0, std_noise_x, len(x))
    noise_v = np.random.normal(0, std_noise_v, len(x))
    return noise_x + x, noise_v + v

In [5]:
def ln_likelihood(theta, data, dt_model):
    chi2 = 0
    k, x0, v0, *t_obs = theta
    x_obs, v_obs, sigma_x, sigma_v = data
    x, v, _, _ = leapfrog(x0, v0, np.array(t_obs), potential_and_grad_py, dt_model, k=k)
    chi2 += -(v - v_obs)**2 / sigma_v**2 - 2*np.log(sigma_v)
    chi2 += -(x - x_obs)**2 / sigma_x**2 - 2*np.log(sigma_x)
    return 0.5*chi2.sum()

In [6]:
def potential_and_grad_py(position, k=1.0):
    #function that returns the potential and it's gradient at a given position
    return 0.5 * k * position**2, k*position

def leap(position, momentum, grad, potential_and_grad, step_size, k=1.0):
    momentum -= 0.5 * step_size * grad
    position += step_size * momentum
    potential, grad = potential_and_grad_py(position, k=k)
    momentum -= 0.5 * step_size * grad
    return position, momentum, potential, grad

def leapfrog(x0, v0, t_obs, potential_and_grad_py, dt, k=np.float64(1.0)):
    #function that takes initial conditions that takes us to the next position 
    x = [] 
    v = [] 
    t = [] 
    grads = []
    time = []
    
    tprime = np.float64(0.0)
    xprime = np.float64(x0)
    vprime = np.float64(v0)
    pot, grad = potential_and_grad_py(xprime, k=k)
    for to in t_obs:

        while tprime + dt < to:
            xprime, vprime, pot, grad = leap(xprime, vprime, grad, potential_and_grad_py, dt, k=k)
            tprime = tprime + dt    
        dt_tiny = to - tprime
        xsave, vsave, potsave, gradsave = leap(xprime, vprime, grad, potential_and_grad_py, dt_tiny, k=k)
        tsave = tprime + dt_tiny
        #print(xsave, vsave, tsave, potsave, gradsave)
        x.append(xsave.copy())
        v.append(vsave.copy())
        t.append(tsave.copy())
        grads.append(grad)
        time.append(tprime)
        #print(x, v)
    return np.array(x), np.array(v), np.array(grads), np.array(time) #, np.array(t)

In [7]:
np.random.seed(1)

## Set some true values and initial guesses

In [8]:
x0_true   = 100.
v0_true   = 100.
k_true    = 10.

#define step size of each leap and number of shos
s_size = np.float64(0.01)      #resolution of each leap
n_shos = 1            #number of simple harmonic oscillators 

#set spring constant initial guess
current_k_value = np.float64(10.0)

#generate initial velocities and momenta guesses
current_x_value = np.float64(10.0) #np.float64(np.random.randn(n_shos))
current_v_value = np.float64(4.0) #np.float64(np.random.randn(n_shos))

max_time  = np.float64(10.)
nobspoints = 10
#t_obs_true = np.linspace(0, max_time, nobspoints) 
t_obs_true = np.random.uniform(0, max_time, nobspoints)
t_obs_true.sort()
#generate initial times to model SHO, for now set equal to the true times
current_t_obs_values = t_obs_true #np.random.uniform(0, max_time, nobspoints)#t_obs_init.sort()

std_noise_x = 10.0
std_noise_v = 10.0
plot_xerr = np.zeros(nobspoints) + std_noise_x
plot_yerr = np.zeros(nobspoints) + std_noise_v


## Generate fake data and model predictions using python implementation (which I trust)

In [9]:
#generate true values and noisify them
x_true, v_true, grad_true, time_steps_true = leapfrog(x0_true, v0_true, t_obs_true, potential_and_grad_py, s_size, k=k_true)
x_obs, v_obs = genData(x_true, v_true, nobspoints, std_noise_x, std_noise_v)

In [10]:
#generate values using python leapfrog from initial guesses to compare with tf model
xleap, vleap, gradleap, timeleap = leapfrog(current_x_value,
                                            current_v_value,
                                            current_t_obs_values,
                                            potential_and_grad_py,
                                            s_size,
                                            k=current_k_value)

In [11]:
#calculate log likelihood of initial guesses using python leapfrog model
theta = [current_k_value, current_x_value, current_v_value] +  current_t_obs_values.tolist()
data = [x_obs, v_obs, std_noise_x, std_noise_v]
print(ln_likelihood(theta, data, s_size))

-2081.361073961145


def evolve(params, t):
    position_eval, momentum_eval, k, grad_eval, potential_eval, t_previous, x_eval, v_eval = params
    print(t, t_previous)
    deltat = t - t_previous
    print(deltat)
    new_p_eval, new_m_eval, new_p_eval, new_g_eval, new_x_eval, new_v_eval, new_time_step_eval = session.run(
      [position_model, momentum_model, potential_model, grad_model, x_model, v_model, time_on_step_model],
      feed_dict = {position: position_eval, momentum: momentum_eval, k: k, dt: deltat})#, grad:grad_val})
    new_t_previous = t_previous + new_time_step_eval

    params = [new_p_eval, new_m_eval, k, new_g_eval, new_p_eval, new_t_previous, new_x_val, new_v_val]
    return params

## Now try with TensorFlow

In [24]:
def potential_and_grad(position, k=np.float64(1.0)):
    #function that returns the potential and it's gradient at a given position
    return 0.5 * k * tf.square(position), k * position

def tf_log_like(x0_input, v0_input, k_input, t_obs_input, obsx, obsv, sigmax, sigmav):
    with tf.variable_scope('Main', reuse=tf.AUTO_REUSE): 
        #assign parameters to model tensors
        tf.assign(current_x, x0_input)
        tf.assign(current_v, v0_input)
        tf.assign(k, k_input)
        tf.assign(t_obs, t_obs_input)
        tf.assign(t_previous, tf.constant(np.float64(0.0)))
        #calculate potential at initial position
        current_potential_value, current_grad_value = potential_and_grad(current_x, k=k)
        tf.assign(current_grad, current_grad_value)

        #loop through observations times and calculate x, v
        for i in range(10):
            #calculate time to evolve for current iteration
            tf.assign(dt, t_obs[i] - t_previous)

            #run the tf model
            new_x, new_v, new_grad, new_t, x_value, v_value = model_leap
            print(new_x, new_v, new_grad, new_t, x_value, v_value)
            #update tf tensors with new values and save x, v
            tf.assign(x[i], x_value)
            tf.assign(v[i], v_value)
            tf.assign(current_x, new_x)
            tf.assign(current_v, new_v)
            tf.assign(current_grad, new_grad)
            tf.assign(t_previous, new_t + t_previous)

            #save a couple other things for debugging
            #time_step[i] = t_previous
            #grads[i] = new_grad

        #calculate loglikelihood for these model parameters
        chi2 = np.float64(0.0)
        chi2 += (v - obsv)**2 / sigmav**2 + 2*tf.log(sigmav)
        chi2 += (x - obsx)**2 / sigmax**2 + 2*tf.log(sigmax)

    return -0.5*tf.reduce_sum(chi2)

"""
def tf_log_like(x0_input, v0_input, k_input, t_obs_input, obsx, obsv, sigmax, sigmav):
    with tf.Session() as session:
        with tf.variable_scope('Main', reuse=tf.AUTO_REUSE): 
            #assign parameters to model tensors
            session.run(tf.global_variables_initializer())
            session.run(tf.assign(current_x, x0_input))
            session.run(tf.assign(current_v, v0_input))
            session.run(tf.assign(k, k_input))
            session.run(tf.assign(t_obs, t_obs_input))
            session.run(tf.assign(t_previous, tf.constant(np.float64(0.0))))
            #calculate potential at initial position
            current_potential_value, current_grad_value = session.run([potential_model, grad_model])
            tf.assign(current_grad, np.float64(current_grad_value))

            #loop through observations times and calculate x, v
            for i in range(nobspoints):
                #calculate time to evolve for current iteration
                session.run(tf.assign(dt, t_obs[i] - t_previous))
                
                #run the tf model
                new_x, new_v, new_grad, new_t, x_value, v_value = model_leap

                #update tf tensors with new values and save x, v
                session.run(tf.assign(x[i], x_value))
                session.run(tf.assign(v[i], v_value))
                session.run(tf.assign(current_x, new_x))
                session.run(tf.assign(current_v, new_v))
                session.run(tf.assign(current_grad, new_grad))
                session.run(tf.assign(t_previous, new_t + t_previous))
                
                #save a couple other things for debugging
                time_step[i] = session.run(t_previous)
                grads[i] = new_grad
            
            #calculate loglikelihood for these model parameters
            chi2 = np.float64(0.0)
            chi2 += (v - obsv)**2 / sigmav**2 + 2*tf.log(sigmav)
            chi2 += (x - obsx)**2 / sigmax**2 + 2*tf.log(sigmax)

    return -0.5*tf.reduce_sum(chi2)
"""


tf.reset_default_graph()

#=============================================================
#=======  SETTING UP VARIABLES ===============================
#=============================================================

#the time resolution of the integrator
step_size = tf.constant(s_size, dtype=np.float64, name='step_size') #tf.placeholder(np.float32, name='step_size')
i         = tf.constant(np.float64(0.0), name='i')

#turn fake data into tensorflow tensors 
x_obs_tf = tf.constant(x_obs, dtype=np.float64)
v_obs_tf = tf.constant(v_obs, dtype=np.float64)
std_noise_x_tf = tf.constant(std_noise_x, dtype=np.float64)
std_noise_v_tf = tf.constant(std_noise_v, dtype=np.float64)


with tf.variable_scope("Main"):

    #create placeholders for initial position and momentum, spring constant, and obs times to define model
    #these are the model parameters the optimize will find
    current_x    = tf.Variable(tf.constant(current_x_value), name = "current_x", dtype=tf.float64)
    current_v    = tf.Variable(tf.constant(current_v_value), name = "current_v", dtype=np.float64)
    current_grad = tf.Variable(tf.constant(np.float64(0.0)), name = "current_grad", dtype=np.float64)
    
    x0           = tf.Variable(tf.constant(current_x_value), name = "current_x", dtype=tf.float64)
    v0           = tf.Variable(tf.constant(current_v_value), name = "current_v", dtype=np.float64)
    k            = tf.Variable(tf.constant(current_k_value), name = "k"        , dtype=np.float64)
    t_obs        = tf.Variable(tf.constant(current_t_obs_values), name = "t_obs"   , dtype=np.float64)    

    #the time of the previous observation to help us calculate dt
    t_previous = tf.Variable(tf.constant(np.float64(0.0)), name = 't_previous', dtype=tf.float64)
    t_current  = tf.Variable(tf.constant(np.float64(1.0)), name = 't_current',  dtype=tf.float64)
    #the time between the last observation time and the next
    #so how long the current iteration should integrate to get to the next observation
    #I'm a bit confused about when to use a placeholder and when to use a variable
    #this is something I will always feed into the model and will change with each iteration
    dt       = tf.Variable(tf.constant(np.float64(1.0)), name = "dt"        , dtype=np.float64)


    #the modeled observed positions and velocities compare with the actual observed positions and velocities for the loss function 
    x         = tf.Variable(tf.zeros(nobspoints, dtype=np.float64), name = "x", dtype=tf.float64)
    v         = tf.Variable(tf.zeros(nobspoints, dtype=np.float64), name = "v", dtype=tf.float64)
    nobs      = tf.Variable(tf.constant(nobspoints), name='nobs')

#generate arrays to save values from model for debugging purposes 
grads     = np.zeros([nobspoints, n_shos])
time_step = np.zeros([nobspoints, n_shos])



#=============================================================
#=======  DEFINING MODELS ====================================
#=============================================================

with tf.variable_scope('Main', reuse=tf.AUTO_REUSE):

    #define potential model
    potential_model, grad_model = potential_and_grad(current_x, k=k)

    #define model, leapfrog_integrator is from hmc tensorflow stuff
    model_leap = tflf.leapfrog_integrator(step_size, dt, 
                                      current_x, current_v, 
                                      potential_and_grad, 
                                      current_grad, t_previous, k=k)

    #define log likelihood model
    #ll = tf_log_like(x, v, x_obs_tf, v_obs_tf, std_noise_x_tf, std_noise_v_tf)
    ll = tf_log_like(x0, v0, k, t_obs, x_obs_tf, v_obs_tf, std_noise_x_tf, std_noise_v_tf)

    #define gradients 
    gradients = tf.gradients(ll, [x0, v0]) #, current_v, k])

#=============================================================
#=======  RUN TENSORFLOW =====================================
#=============================================================

with tf.Session() as session:
    with tf.variable_scope('Main', reuse=tf.AUTO_REUSE):  
        session.run(tf.global_variables_initializer())
        print([v.name for v in tf.all_variables()])
        # This step is needed to set up the variables.


        print("The log likelihood computed using tensorflow: {0}"
              .format(session.run(ll)))
        grad_tf = session.run(gradients)
        #print("The gradients are: {0}".format(session.run(gradients)))
        
        fig, ax = plt.subplots(1,4, figsize=(10, 2.5))
        ax[0].plot(current_t_obs_values, session.run(x) - xleap, 'o')
        #print(session.run(x))
        ax[0].set_xlabel('t')
        ax[0].set_ylabel('$\Delta x$')
        ax[1].set_xlabel('t')
        ax[1].set_ylabel('$\Delta y$')
        ax[1].plot(current_t_obs_values, session.run(v) - vleap, 'o')
        ax[2].set_xlabel('t')
        ax[2].set_ylabel('$\Delta \Phi$')
        ax[2].plot(current_t_obs_values, grads.T[0] - gradleap.T, 'o')
        ax[3].set_xlabel('t')
        ax[3].set_ylabel('$\Delta t$')
        ax[3].plot(current_t_obs_values, time_step.T[0] - timeleap.T, 'o')
        plt.tight_layout()
        
        fig, ax = plt.subplots(1,4, figsize=(10, 2.5))
        ax[0].plot(current_t_obs_values, xleap, 'o')
        ax[0].plot(current_t_obs_values, session.run(x), 'o')
        ax[0].set_xlabel('t')
        ax[0].set_ylabel('x')
        ax[1].set_xlabel('t')
        ax[1].set_ylabel('y')
        ax[1].plot(current_t_obs_values, vleap, 'o')
        ax[1].plot(current_t_obs_values, session.run(v), 'o')
        
ax[2].set_xlabel('t')
ax[2].set_ylabel('gradient of potential')
ax[2].plot(current_t_obs_values, gradleap, 'o')
ax[2].plot(current_t_obs_values, grads, 'o')
ax[3].set_xlabel('t')
ax[3].set_ylabel('time at step')
ax[3].plot(current_t_obs_values, time_step, 'o')
ax[3].plot(current_t_obs_values, timeleap, 'o')
plt.legend()
plt.tight_layout()

Tensor("Main_1/leapfrog_integrator/while/Exit_2:0", shape=(), dtype=float64) Tensor("Main_1/leapfrog_integrator/while/Exit_3:0", shape=(), dtype=float64) Tensor("Main_1/leapfrog_integrator/mul_2:0", shape=(), dtype=float64) Tensor("Main_1/mul_3:0", shape=(), dtype=float64) Tensor("Main_1/leapfrog_integrator/leapfrog_step/add:0", shape=(), dtype=float64) Tensor("Main_1/leapfrog_integrator/leapfrog_step/sub_1:0", shape=(), dtype=float64)
Tensor("Main_1/leapfrog_integrator/while/Exit_2:0", shape=(), dtype=float64) Tensor("Main_1/leapfrog_integrator/while/Exit_3:0", shape=(), dtype=float64) Tensor("Main_1/leapfrog_integrator/mul_2:0", shape=(), dtype=float64) Tensor("Main_1/mul_3:0", shape=(), dtype=float64) Tensor("Main_1/leapfrog_integrator/leapfrog_step/add:0", shape=(), dtype=float64) Tensor("Main_1/leapfrog_integrator/leapfrog_step/sub_1:0", shape=(), dtype=float64)
Tensor("Main_1/leapfrog_integrator/while/Exit_2:0", shape=(), dtype=float64) Tensor("Main_1/leapfrog_integrator/while/Ex

TypeError: Fetch argument None has invalid type <class 'NoneType'>

In [ ]:
xleap

In [ ]:
import tensorflow as tf

def scoping(fn, scope1, scope2, vals):
    with fn(scope1):
        a = tf.Variable(vals[0], name='a')
        b = tf.get_variable('b', initializer=vals[1])
        c = tf.constant(vals[2], name='c')

        with fn(scope2):
            d = tf.add(a * b, c, name='res')

        print('\n  '.join([scope1, a.name, b.name, c.name, d.name]), '\n')
    return d

d1 = scoping(tf.variable_scope, 'scope_vars', 'res', [1, 2, 3])
d2 = scoping(tf.name_scope,     'scope_name', 'res', [1, 2, 3])

with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(tf.global_variables_initializer())
    print(sess.run([d1, d2]))
    writer.close()

# License: See LICENSE
# Fit a straight line, of the form y=m*x+b

import tensorflow as tf
import numpy as np

'''
Your dataset.
'''
xs = np.linspace(0.0, 8.0, 8000000) # 8-million features
ys = 0.3*xs-0.8+np.random.normal(scale=0.25, size=len(xs)) # 8-million labels

'''
Initial guesses, which will be refined by TensorFlow.
'''
m_initial = -0.5 # Initial guesses
b_initial =  1.0

'''
Define free variables to be solved.
'''
m = tf.Variable(m_initial) # Parameters
b = tf.Variable(b_initial)

'''
Define placeholders for big data.
'''
_BATCH = 8 # Use only eight points at a time.
xs_placeholder = tf.placeholder(tf.float32, [_BATCH])
ys_placeholder = tf.placeholder(tf.float32, [_BATCH]) 

'''
Define the error between the data and the model as a tensor (distributed computing).
'''
ys_model = m*xs_placeholder+b # Tensorflow knows this is a vector operation
total_error = tf.reduce_sum((ys_placeholder-ys_model)**2) # Sum up every item in the vector

'''
Once cost function is defined, create gradient descent optimizer.
'''
optimizer_operation = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(total_error) # Does one step

'''
Create operator for initialization.
'''
initializer_operation = tf.global_variables_initializer()

'''
All calculations are done in a session.
'''
with tf.Session() as session:

	session.run(initializer_operation) # Call operator

	_EPOCHS = 10000 # Number of "sweeps" across data
	for iteration in range(_EPOCHS):
		random_indices = np.random.randint(len(xs), size=_BATCH) # Randomly sample the data
		feed = {
			xs_placeholder: xs[random_indices],
			ys_placeholder: ys[random_indices]
		}
		session.run(optimizer_operation, feed_dict=feed) # Call operator

	slope, intercept = session.run((m, b)) # Call "m" and "b", which are operators
	print('Slope:', slope, 'Intercept:', intercept)

In [ ]:
print(autograph.to_code(leapfrog))

In [ ]:
        #calculate potential and gradient at initial position
        #assign current_grad_value to current_grad model 
        current_potential_value, current_grad_value = session.run([potential_model, grad_model])

        tf.assign(current_grad, np.float64(current_grad_value))
        #print('all obs times: ', session.run(t_obs))
        for i in range(nobspoints):
            t_current = t_obs[i]
            deltat = t_current - t_previous
            #print('current time: ', session.run(t_current))
            session.run(tf.assign(dt, deltat))
            new_x, new_v, new_grad, new_t, x_value, v_value = session.run(model_leap)
            session.run(tf.assign(x[i], x_value))
            session.run(tf.assign(v[i], v_value))
            session.run(tf.assign(current_x, new_x))
            session.run(tf.assign(current_v, new_v))
            session.run(tf.assign(current_grad, new_grad))
            session.run(tf.assign(t_previous, new_t + t_previous))
            #print('current dt: ', session.run(dt))
            time_step[i] = session.run(t_previous)
            grads[i] = new_grad
            #print('current x: ', session.run(current_x))
            #print('current v: ', session.run(current_v))
            #print('current grad: ', session.run(current_grad))
        """
        #define loop over t_obs, inside loop assigns values to x, v
        #with tf.name_scope(None, 'evolve'):
            #print(nobspoints)
            count = tf.scan(evolve, np.arange(nobspoints), 
                              initializer=(tf.constant(np.float64(0.0), dtype=np.float64), 
                                           tf.constant(np.float64(0.0), dtype=np.float64),
                                           tf.constant(np.float64(0.0), dtype=np.float64),
                                           tf.constant(np.float64(0.0), dtype=np.float64),
                                           tf.constant(np.float64(0.0), dtype=np.float64),
                                           tf.constant(np.float64(0.0), dtype=np.float64)))
                                           #tf.constant(np.int32(0.0))])
                                              #[tf.constant(np.int32(0.0))])
            print(session.run(count))
        #loop = tf.map_fn(evolve, iteration) #, initializer=tf.constant(np.float64(0.0)))

        #session.run(loop)
        with tf.variable_scope('Main', reuse=tf.AUTO_REUSE):
        """
def counter_fn(counter):  # pylint: disable=unused-argument
    return counter  < 10

def evolve(params, i):
    print(i)
    _, _, _, t_previous, _, _ = params
    with tf.variable_scope('Main', reuse=tf.AUTO_REUSE):
        tcurrent = tf.get_variable("t_obs", [1], dtype=np.float64)[i]
        dt = tcurrent - t_previous 
        new_x, new_v, new_grad, new_t, x_value, v_value = model_leap

    return new_x, new_v, new_grad, new_t, x_value, v_value

